# **Model**

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
df_train = pd.read_csv("data/train.csv")

In [12]:
df_train

,Unnamed: 0,order_value_gbp,restaurant_id,number_of_items,prep_time_seconds,country,city,type_of_food,day,hour
0,0,59.90000,1326,2,6220,UK,London,burritos,1,12
1,1,24.00000,1326,8,603,UK,London,burritos,6,17
2,2,15.25000,1326,3,443,UK,London,burritos,8,14
3,3,28.05000,1326,8,669,UK,London,burritos,12,15
4,4,56.30000,255,7,3314,UK,London,chinese,1,13
...,...,...,...,...,...,...,...,...,...,...
32389,32389,35.15000,3185,4,913,UK,Glasgow,italian,30,21
32390,32390,9.15771,3556,3,201,Germany,Berlin,vietnamese,30,22
32391,32391,18.20000,3517,3,863,UK,Cheltenham,indian,30,21
32392,32392,21.50000,1705,3,1339,UK,London,italian,30,22


In [7]:
# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990842, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06832974, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286377, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04687948,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452837, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00421986,  0.00306441]])